In [1]:
import numpy
import datetime as dt
from astral import LocationInfo
from astral.location import Location
from astral.geocoder import database, group, lookup_in_group
import pytz

In [2]:
# calculate sunrise for Berlin
# name and region can be any value you like
city=LocationInfo(name="Berlin", region="Germany", timezone="Europe/Berlin", 
                  latitude=52.547302, longitude=13.390178)

In [3]:
berlin = Location(city)

In [4]:
berlin.golden_hour()

(datetime.datetime(2021, 2, 23, 6, 43, 57, 949737, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 datetime.datetime(2021, 2, 23, 7, 52, 36, 472881, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>))

In [9]:
# Sun dates for tomorrow
sun_params = berlin.sun(date=dt.date(2021, 2, 22))
sun_params

{'dawn': datetime.datetime(2021, 2, 22, 6, 32, 53, 728316, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'sunrise': datetime.datetime(2021, 2, 22, 7, 8, 17, 673058, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'noon': datetime.datetime(2021, 2, 22, 12, 19, 57, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'sunset': datetime.datetime(2021, 2, 22, 17, 32, 20, 225256, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>),
 'dusk': datetime.datetime(2021, 2, 22, 18, 7, 48, 853790, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>)}

In [11]:
# convert datetime object to google cal iso format
sun_params['dawn'].isoformat()

'2021-02-22T06:32:53.728316+01:00'

In [7]:
# this is a dataclass defined in __init__.py!
type(city)

astral.LocationInfo

## Obtain location info from db of this package

In [54]:
# imports the database of cities and associated timezones, longitude & latitude and so on
db=database()

In [20]:
# dict with keys being European cities and the values are LocationInfo objects!!
europe = group('Europe', db)

In [21]:
lookup_in_group('Berlin', europe)

LocationInfo(name='Berlin', region='Germany', timezone='Europe/Berlin', latitude=52.5, longitude=13.416666666666666)

In [26]:
europe['berlin']

[LocationInfo(name='Berlin', region='Germany', timezone='Europe/Berlin', latitude=52.5, longitude=13.416666666666666)]

In [28]:
db.keys()

dict_keys(['asia', 'africa', 'australia', 'europe', 'indian', 'pacific', 'america', 'etc', 'brazil', 'utc', 'us', 'atlantic'])